# Import Statemets and EE Initialisation

In [ ]:
import os
import ee
from google.colab import drive
from google.colab import files
from datetime import time
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)


ee.Authenticate(force=True)
ee.Initialize(project='')

# Runoff Computation using Curve Number (CN) method

In [ ]:
START_DATE = ''
END_DATE = ''
START_YEAR = '2020-01-01'
END_YEAR = '2020-12-31'
OUTPUT_FOLDER_PATH = ''
soil = ee.Image('projects/ee-dharmisha-siddharth/assets/HYSOGs250m')
shape = ee.FeatureCollection('projects/ee-ankit-mcs/assets/angul_ext_boundary')
srtm2 = ee.Image("USGS/SRTMGL1_003")
dem = ee.Image('projects/ee-ankit-mcs/assets/depressionless-DEM/Brahmani')

geometry = shape.geometry()

DEM = srtm2.clip(geometry)
slope = ee.Terrain.slope(DEM)
depressed = dem.clip(geometry)
# Exporting the Depressionless DEM
task1 = ee.batch.Export.image.toDrive(
    image=depressed,
    description='',
    folder=OUTPUT_FOLDER_PATH,
    region=shape.geometry().bounds(),
    scale=30,
    maxPixels=1e13
)
task1.start()

lulc=ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(START_YEAR, END_YEAR)
classification = lulc.select('label')
dwComposite = classification.reduce(ee.Reducer.mode());
dwComposite = dwComposite.clip(shape).rename('label').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )
lulc = dwComposite.rename(['lulc'])
soil = soil.expression(
    "(b('b1') == 14) ? 4" +
      ": (b('b1') == 13) ? 3" +
        ": (b('b1') == 12) ? 2" +
           ": (b('b1') == 11) ? 1" +
             ": b('b1')"
).rename('soil')
soil = soil.clip(shape).rename('soil').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )
lulc_soil = lulc.addBands(soil)
lulc_soil=lulc_soil.unmask(0)
CN2 = lulc_soil.expression(
    "(b('soil') == 1) and(b('lulc')==0) ? 0" +
     ": (b('soil') == 1) and(b('lulc')==1) ? 30" +
        ": (b('soil') == 1) and(b('lulc')==2) ? 39" +
        ": (b('soil') == 1) and(b('lulc')==3) ? 0" +
        ": (b('soil') == 1) and(b('lulc')==4) ? 64" +
        ": (b('soil') == 1) and(b('lulc')==5) ? 39" +
        ": (b('soil') == 1) and(b('lulc')==6) ? 82" +
        ": (b('soil') == 1) and(b('lulc')==7) ? 49" +

          ": (b('soil') == 2) and(b('lulc')==0) ? 0" +
          ": (b('soil') == 2) and(b('lulc')==1) ? 55" +
          ": (b('soil') == 2) and(b('lulc')==2) ? 61" +
          ": (b('soil') == 2) and(b('lulc')==3) ? 0" +
          ": (b('soil') == 2) and(b('lulc')==4) ? 75" +
          ": (b('soil') == 2) and(b('lulc')==5) ? 61" +
          ": (b('soil') == 2) and(b('lulc')==6) ? 88" +
          ": (b('soil') == 2) and(b('lulc')==7) ? 69" +

            ": (b('soil') == 3) and(b('lulc')==0) ? 0" +
            ": (b('soil') == 3) and(b('lulc')==1) ? 70" +
            ": (b('soil') == 3) and(b('lulc')==2) ? 74" +
            ": (b('soil') == 3) and(b('lulc')==3) ? 0" +
            ": (b('soil') == 3) and(b('lulc')==4) ? 82" +
            ": (b('soil') == 3) and(b('lulc')==5) ? 74" +
            ": (b('soil') == 3) and(b('lulc')==6) ? 91" +
            ": (b('soil') == 3) and(b('lulc')==7) ? 79" +

            "  : (b('soil') == 4) and(b('lulc')==0) ? 0" +
              ": (b('soil') == 4) and(b('lulc')==1) ? 77" +
              ": (b('soil') == 4) and(b('lulc')==2) ? 80" +
              ": (b('soil') == 4) and(b('lulc')==3) ? 0" +
              ": (b('soil') == 4) and(b('lulc')==4) ? 85" +
              ": (b('soil') == 4) and(b('lulc')==5) ? 80" +
              ": (b('soil') == 4) and(b('lulc')==6) ? 93" +
              ": (b('soil') == 4) and(b('lulc')==7) ? 84" +
                   ": (b('soil') == 0) ? 0" +
                  ": 0"
).rename('CN2')

CN2 = CN2.clip(shape).rename('CN2').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

CN1 = CN2.expression(
    '-75*CN2/(CN2-175)',{
      'CN2': CN2.select('CN2')
    }).rename('CN1')

CN1 = CN1.clip(shape).rename('CN1').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

CN3 = CN2.expression(
    'CN2*((2.718)**(0.00673*(100-CN2)))',{
      'CN2': CN2.select('CN2')
    }).rename('CN3')

CN3 = CN3.clip(shape).rename('CN3').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

slope=slope.rename("slope")

slope = slope.clip(shape).rename('slope').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

part1=CN3.select('CN3').subtract(CN2.select('CN2')).divide(ee.Number(3)).rename("p1")

part1 = part1.clip(shape).rename('p1').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

part2 = slope.expression(
    '1-(2*(2.718)**(-13.86*slope))',{
      'slope': slope.select('slope')
    }).rename('p2')

part2 = part2.clip(shape).rename('p2').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

CN2a = slope.expression(
    'p1*p2+CN2',{
      'p1': part1.select('p1'),
      'p2': part2.select('p2'),
      'CN2': CN2.select('CN2')
    }).rename('CN2a')

CN2a = CN2a.clip(shape).rename('CN2a').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )


CN1a = CN2a.expression(
    '4.2*CN2a/(10-0.058*CN2a)',{
      'CN2a': CN2a.select('CN2a')
    }).rename('CN1a')

CN1a = CN1a.clip(shape).rename('CN1a').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )


CN3a = CN2a.expression(
    '23*CN2a/(10+0.13*CN2a)',{
      'CN2a': CN2a.select('CN2a')
    }).rename('CN3a')

CN3a = CN3a.clip(shape).rename('CN3a').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

sr1 = CN1a.expression(
    '(25400/CN1a)-254', {
      'CN1a': CN1a.select('CN1a')
}).rename('sr1')

sr1 = sr1.clip(shape).rename('sr1').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

sr2 = CN2a.expression(
    '(25400/CN2a)-254', {
      'CN2a': CN2a.select('CN2a')
}).rename('sr2')

sr2 = sr2.clip(shape).rename('sr2').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )

sr3 = CN3a.expression(
    '(25400/CN3a)-254', {
      'CN3a': CN3a.select('CN3a')
}).rename('sr3')

sr3 = sr3.clip(shape).rename('sr3').reproject(
  crs = 'EPSG:4326',
  scale = 30
  )



In [ ]:

runoffs=ee.List([])
precip=ee.List([])
base = ee.Date(END_DATE)

def ant(i):
    a = ee.Number(i).multiply(ee.Number(-1))
    b = (ee.Number(i).multiply(ee.Number(-1))).subtract(ee.Number(1))
    c = (ee.Number(i).multiply(ee.Number(-1))).subtract(ee.Number(4))
    dtTo = base.advance(a, 'day').format('YYYY-MM-dd')
    dtMid = base.advance(b, 'day').format('YYYY-MM-dd')
    dtFrom = base.advance(c, 'day').format('YYYY-MM-dd')
    dataset = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational').filter(ee.Filter.date(dtFrom, dtTo))
    antecedent = dataset.reduce(ee.Reducer.sum());
    antecedent = antecedent.clip(shape).select('hourlyPrecipRate_sum').reproject(
        crs = 'EPSG:4326',
        scale = 30
        )

    M2 = CN2a.expression(
        '0.5*(-sr+sqrt((sr**2)+(4*p*sr)))', {
          'sr': sr2.select('sr2'),
          'p':antecedent.select('hourlyPrecipRate_sum')
    }).rename('m2')

    M2 = M2.clip(shape).rename('m2').reproject(
      crs = 'EPSG:4326',
      scale = 30
      )
    M1 = CN2a.expression(
        '0.5*(-sr+sqrt((sr**2)+(4*p*sr)))', {
          'sr': sr1.select('sr1'),
          'p':antecedent.select('hourlyPrecipRate_sum')
    }).rename('m1')

    M1 = M1.clip(shape).rename('m1').reproject(
      crs = 'EPSG:4326',
      scale = 30
      )

    M3 = CN2a.expression(
        '0.5*(-sr+sqrt((sr**2)+(4*p*sr)))', {
          'sr': sr3.select('sr3'),
          'p':antecedent.select('hourlyPrecipRate_sum')
    }).rename('m3')

    M3 = M3.clip(shape).rename('m3').reproject(
      crs = 'EPSG:4326',
      scale = 30
      )

    dataset = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational').filter(ee.Filter.date(dtMid, dtTo));
    total = dataset.reduce(ee.Reducer.sum());

    total = total.clip(shape).select('hourlyPrecipRate_sum').reproject(
      crs = 'EPSG:4326',
      scale = 30
      )


    runoff = total.expression(
            '(P>=0.2*sr1) and (P5>=0) and (P5<=35) and (((P-0.2*sr1)*(P-(0.2*sr1)+m1))/(P+(0.2*sr1)+sr1+m1))>=0? ((P-0.2*sr1)*(P-(0.2*sr1)+m1))/(P+(0.2*sr1)+sr1+m1)' +
            ': (P>=0.2*sr2) and (P5>=0) and (P5>35) and (((P-0.2*sr2)*(P-(0.2*sr2)+m2))/(P+(0.2*sr2)+sr2+m2))>=0 ? ((P-0.2*sr2)*(P-(0.2*sr2)+m2))/(P+(0.2*sr2)+sr2+m2)' +
            ': (P>=0.2*sr3) and (P5>=0) and (P5>52.5) and (((P-0.2*sr3)*(P-(0.2*sr3)+m3))/(P+(0.2*sr3)+sr3+m3))>=0 ? ((P-0.2*sr3)*(P-(0.2*sr3)+m3))/(P+(0.2*sr3)+sr3+m3)' +
            ':0',
            {
            'P':total.select('hourlyPrecipRate_sum'),
            'm1':M1.select('m1'),
            'm2':M2.select('m2'),
            'm3':M3.select('m3'),
            'P5':antecedent.select('hourlyPrecipRate_sum'),
            'sr2':sr2.select('sr2'),
            'sr1':sr1.select('sr1'),
            'sr3':sr3.select('sr3')
          }).rename('runoff')
    return runoff

def calculate_runoff(startDate, endDate):
    # Create an empty image to accumulate results.
    combined_runoff = ee.Image()
    # Calculate the number of days for iteration.
    numDays = endDate.difference(startDate, 'days')

    def day_runoff(d, image):
        # Calculate the date.
        a = ee.Number(d).multiply(ee.Number(-1))
        date = endDate.advance(a, 'day')
        dtTo = date.format('YYYY-MM-dd')
        runoff = ant(d)  
        bandName = ee.String('runoff_').cat(dtTo)
        # Combine the runoff image by adding it as a new band. Use the date as the band name.
        return ee.Image(image).addBands(runoff.rename(bandName))

    # Iterate over the number of days, applying the logic to each day and accumulating the result.
    combined_runoff = ee.Image(ee.List.sequence(0, numDays.subtract(1)).iterate(day_runoff, combined_runoff))

    return combined_runoff

# Exporting per day runoff rasters

In [ ]:

startDate = ee.Date(START_DATE)
endDate = ee.Date(END_DATE)
combined_runoff_image = calculate_runoff(startDate, endDate)
combined_runoff_float = combined_runoff_image.toDouble()

task2 = ee.batch.Export.image.toDrive(
      image=combined_runoff_float,
      description='',
      folder=OUTPUT_FOLDER_PATH,
      region=shape.geometry().bounds(),
      scale=30,
      maxPixels=1e13
    )
task2.start()

# Tracking and auto download files from drive

In [ ]:
# Get the list of files in the folder
files_to_download = os.listdir(OUTPUT_FOLDER_PATH)

task2_id = task2.id
while True:
        tasks = ee.batch.Task.list()
        task = next((task for task in tasks if task.id == task2_id), None)
        if task is None:
            print("Task with ID {} not found.".format(task2_id))
            break
        if task.status()['state'] == 'COMPLETED':
            # Task completed, execute the function
            break
        elif task.status()['state'] in ['FAILED', 'CANCELLED']:
            # Task failed or cancelled, handle accordingly
            print("Task {} failed or was cancelled.".format(task2_id))
            break
        else:
            # Task is still running, wait and check again
            print("Task {} is still running...".format(task2_id))
            time.sleep(10)  # Check again after 10 seconds

def check_files_exist(files_list):
    for file_name in files_list:
        file_path = os.path.join(OUTPUT_FOLDER_PATH, file_name)
        if not os.path.isfile(file_path):
            return False
    return True



# Wait until all files are present in the folder
while not check_files_exist(files_to_download):
    print("Waiting for files to be available in the folder...")
    time.sleep(10)  # Wait for 10 seconds before checking again
    drive.mount('/content/drive', force_remount=True)

print("All files are available. Starting download...")



for file_name in files_to_download:
    # Construct the full path of the file
    file_path = os.path.join(OUTPUT_FOLDER_PATH, file_name)

    # Check if it's a file (not a sub-folder)
    if os.path.isfile(file_path):
        # Download the file to your local machine
        files.download(file_path)


# Storing Daily precipitation 

In [ ]:


# Load the exterior boundary polygon (replace 'your_asset_id' with actual asset ID)
boundary = ee.FeatureCollection('projects/ee-ankit-mcs/assets/masalia_mws_12_356033')
# Load the JAXA GSMaP dataset (hourly precipitation rates)
dataset = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational') \
    .filterDate(startDate, endDate) \
    .select('hourlyPrecipRate')  # Select precipitation band

# Function to group images by day and sum precipitation for each day
def daily_precip(date):
    """Sums all hourly precipitation images for a given day."""
    start = ee.Date(date)
    end = start.advance(1, 'day')  # Get the next day

    # Filter hourly images for the current day
    daily_images = dataset.filterDate(start, end)

    # Sum all hourly precipitation images for the day
    daily_sum = daily_images.reduce(ee.Reducer.sum())

    # Rename the band with the date (YYYYMMDD format)
    date_str = start.format('YYYYMMdd')
    return daily_sum.set('system:time_start', start).rename(date_str)

# Generate a list of dates between start and end dates
date_list = ee.List.sequence(0, endDate.difference(startDate, 'day')).map(
    lambda n: startDate.advance(n, 'day')
)

# Map function over the list of dates to generate daily precipitation images
daily_precip_collection = ee.ImageCollection(date_list.map(daily_precip))

# Convert the ImageCollection to a single multi-band raster
multi_band_raster = daily_precip_collection.toBands()

# Clip to the boundary polygon
clipped_raster = multi_band_raster.clip(boundary)

# Export the multi-band raster to Google Drive
task3 = ee.batch.Export.image.toDrive(
    image=clipped_raster,
    description='',
    folder= OUTPUT_FOLDER_PATH,
    scale=30,  # Set resolution to 30m
    region=boundary.geometry().bounds(),
    crs='EPSG:4326',
    maxPixels=1e13
)

# Start the export task
task3.start()

print("Export started. Check Google Drive for the raster file.")
